<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
استخرمون نشه؟</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در بسیاری از وب‌سایت‌های گردشگری و رزرو اقامتگاه‌ها، کاربران می‌توانند تصاویر مختلفی از اقامتگاه‌های مختلف را مشاهده کنند. در این مسئله قصد داریم با استفاده از مجموعه‌ای از تصاویر اقامتگاه‌ها، مدلی طراحی کنیم که بتواند بر اساس تحلیل تصاویر یک اقامتگاه، وجود یا عدم وجود استخر در آن اقامتگاه را پیش‌بینی کند.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [ ]:
import pandas as pd
import numpy as np 

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
مجموعه داده ارائه‌شده به صورت JSON حاوی اطلاعاتی درباره ۱۵۹۹ اقامتگاه است که شامل ویژگی‌های مختلفی می‌باشند. این مجموعه شامل یک لیست با کلید "rooms" است که اقامتگاه‌های موجود در آن قرار دارند. هر رکورد در این مجموعه داده نمایانگر یک اقامتگاه با خصوصیات و امکانات مربوط به آن است. در جدول زیر، کلیدها و توضیحات آن‌ها آورده شده‌اند.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>

|کلید|توضیحات|
|:------:|:---:|
|id|شناسه یکتای اقامتگاه|
|title|عنوان اقامتگاه|
|description|توضیحاتی در مورد اقامتگاه|
|province|اطلاعات استان شامل شناسه و نام استان|
|city|اطلاعات شهر شامل شناسه و نام شهر|

</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
نمونه‌ای از این داده‌ها به صورت زیر است:
</font>
</p>

```json
{
  "rooms": [
    {
      "id": 3202100,
      "title": "اجاره سوئیت در جاده جواهرده - اشکونکوه",
      "description": "این سوئیت بدون اتاق خواب با بالکنی دلباز و چشم اندازی زیبا در طبقه همکف یک ساختمان دو طبقه واقع شده است. فاصله اقامتگاه تا سوپرمارکت و نانوایی به ترتیب حدود ۵۰ و ۵۰۰ متر می باشد.",
      "province": {
        "id": "p26",
        "name": "مازندران"
      },
      "city": {
        "id": 303,
        "name": "رامسر"
      },
    },
    ...
  ]
}
```


<h2 align="right" style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size="3">
    ابتدا نیاز است فایل‌های مجموعه‌‌ی داده را بخوانید. ویژگی‌های آموزشی در فایل <code>train.json</code> و دایرکتوری مربوط به تصاویر نمونه‌های آزمون که باید دسته‌ی آن‌ها را پیش‌بینی کنید، در فایل <code>test.json</code> ذخیره شده‌اند. در صورت نیاز می‌توانید بخشی از داده‌های آموزشی را به عنوان داده‌های اعتبارسنجی جدا کنید. توجه داشته باشید که ستون‌های <code>description</code> و <code>title</code> از داده‌های آزمون حذف شده‌اند و هدف اصلی این فایل، دسترسی به دایرکتوری تصاویر داده‌های آزمون است. دایرکتوری هر تصویر برابر با <code>id</code> هر اقامتگاه است و داخل پوشه <code>pictures</code> قرار دارد.
</font>
</p>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <font face="vazir">
        <span style="color:orange"><b>نکته:</b></span>
        حجم فایل‌های این سوال حدود ۳.۸ گیگابایت است. در صورتی‌که بارگذاری این فایل‌ها در محیط کولب برای‌تان دشوار است می‌توانید با استفاده از دستور زیر این دو فایل را به‌صورت مستقیم در محیط کولب بارگیری کنید.
        همچنین می‌توانید از لینک درایو زیر فایل‌ها را دانلود کرده و بر روی محیط دستگاه خود از آن استفاده کنید.
    </font>
</p>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
    <font face="vazir">
        <a href="https://drive.google.com/file/d/1b8O_a6ywcsbLqJAGDGCkePrdn1cFlXl0/view?usp=sharing" target="_blank" style="color:blue;">لینک دانلود از گوگل درایو</a>
    </font>
</p>





In [ ]:
# # Run this cell to download directly to colab\

# # Install gdown
# !pip install gdown

# # Import necessary libraries
# import gdown
# import zipfile

# # Google Drive file ID and destination filename
# file_id = '1b8O_a6ywcsbLqJAGDGCkePrdn1cFlXl0'
# destination = 'downloaded_file.zip'

# # Download the file from Google Drive
# gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

# # Unpack the zip file
# with zipfile.ZipFile(destination, 'r') as zip_ref:
#     zip_ref.extractall('unzipped_content')

# print("Download and extraction complete.")


In [ ]:
train_data = pd.read_json('directory to train.csv file.')
print(train_data)

In [ ]:
test_data = pd.read_json('directory to test.csv file.')
print(test_data)

<h2 align="right" style="line-height: 200%; font-family: Vazir; color: #0099cc;">
    پیش‌پردازش و مهندسی ویژگی
</h2>

<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
    در این سؤال، باید با استفاده از توصیفات و تیتر هر اقامت‌گاه، وجود یا عدم وجود استخر را استخراج کنید. می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم توسط سامانه داوری ارزیابی نمی‌شوند، بلکه همه آن‌ها بر دقت مدل شما تأثیر خواهند گذاشت. بنابراین، هرچه پیش‌پردازش و مهندسی ویژگی بهتری انجام دهید و دقت مدل بهبود یابد، امتیاز بیشتری از این سؤال کسب خواهید کرد.
</p>


In [ ]:
# extract the existence of a pool from the text

In [ ]:
# do some preprocessing 

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    حال که داده‌ها را پیش‌پردازش کرده‌اید، و دسته‌بندی وجود یا عدم وجود استخر را از متن انجام داده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله یعنی وجود یا عدم وجود استخر را پیش‌بینی کند.
    <br>
    شما مجاز هستید از هر مدلی که آموخته‌اید استفاده کنید. هدف این سوال پیش‌بینی هرچه بهتر متغیر هدف است! که آن همان وجود یا عدم وجود استخر است!
</font>
</p>

In [ ]:
# modeling 

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه داده آموزش یا اعتبارسنجی ارزیابی کنید.
    
</font>
</p>

In [ ]:
# evaluate model
from sklearn.metrics import  f1_score, classification_report, confusion_matrix


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پس از مهندسی ویژگی و مدلسازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در داده تست استفاده کنید و نتایج را در قالب جدول (<code>dataframe</code>) زیر آماده کنید.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|------|---|
|Pool|وجود یا عدم وجود استخر|
|id|شناسه یکتای اقامتگاه|
    
</font>
</div>



<p dir="rtl" style="direction: rtl; text-align: justify; line-height: 200%; font-family: Vazir; font-size: medium;">
    نام دیتافریم باید <i>submission</i> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش شما را ارزیابی کند.
    <br>
    این دیتافریم شامل دو ستون با نام‌های <i>id</i> و <i>Pool</i> است و دارای ۸۶۲ سطر می‌باشد.
    <br>
    مقادیر موجود در ستون <i>Pool</i> شامل <i>True</i> یا <i>False</i> هستند.
    <br>
    برای هر سطر موجود در دیتافریم <i>test</i> باید یک مقدار پیش‌بینی‌شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، مقادیر ستون <i>Pool</i> ممکن است متفاوت باشد.
</p>

<div style="text-align: center; line-height: 200%; font-family: Vazir; font-size: medium;">
    <font face="vazir" size="3">
        <table border="1" style="margin: 0 auto;">
            <tr>
                <th>id</th>
                <th>Pool</th>
            </tr>
            <tr>
                <td>3160664</td>
                <td>True</td>
            </tr>
            <tr>
                <td>3195184</td>
                <td>False</td>
            </tr>
            <tr>
                <td>3224078</td>
                <td>False</td>
            </tr>
            <tr>
                <td>3233712</td>
                <td>True</td>
            </tr>
            <tr>
                <td>3201449</td>
                <td>True</td>
            </tr>
        </table>
    </font>
</div>

In [ ]:
# predict test samples
submission = pd.DataFrame()
submission


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [ ]:
import zipfile
import joblib

if not os.path.exists(os.path.join(os.getcwd(), 'EstakhtemoonNasheh.ipynb')):
    %notebook -e EstakhtemoonNasheh.ipynb

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)
            
submission.to_csv('submission.csv', index=False)
file_names = ['EstakhtemoonNasheh.ipynb', 'submission.csv']
compress(file_names)